### Producing Sentiment variations using LLMs

Author: Raphael Merx
Input: a baseline sentence; Output: variations of this sentence where a target word has more positive or negative sentiment


In [1]:
#pip install simplemind python-dotenv

In [2]:
from dotenv import load_dotenv
load_dotenv()

from dataclasses import dataclass
import simplemind as sm

TARGET_WORD = 'anxiety'
#TARGET_WORDS = ['abuse', 'anxiety', 'depression', 'mental_health', 'mental_illness', 'trauma']

# can be changed to gemini, see https://pypi.org/project/simplemind/
PROVIDER = "openai"
MODEL = "gpt-4o"

Get neutral sentences from corpus for LLM input for each target

**Aim**: This script processes text files containing sentences for specified target terms, calculates sentence-level mean sentiment scores using the NRC-VAD lexicon, identifies 
sentences with neutral arousal scores (dynamically globally determined from NRC-VAD dataset), and saves these sentences along with their metadata to output files in the output folder.

In [6]:
%run step0_get_neutral_baselines_sentiment.py

c:\Users\naomi\anaconda3\Lib\site-packages\pandas\core\arrays\masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


Dynamic neutral sentiment range: (0.5, 0.521)
Processing file: c:\Users\naomi\OneDrive\COMP80004_PhDResearch\RESEARCH\PROJECTS\3_evaluation+validation - ACL 2025\sentiment-breadth-intensity\0.0_corpus_preprocessing\output\natural_lines_targets\abuse.lines.psych
Saved neutral sentiment sentences for abuse to c:\Users\naomi\OneDrive\COMP80004_PhDResearch\RESEARCH\PROJECTS\3_evaluation+validation - ACL 2025\sentiment-breadth-intensity\1_sentiment\synthetic\input\baselines\abuse_neutral_baselines_sentiment.csv.
Total neutral sentences for abuse: 4538
Processing file: c:\Users\naomi\OneDrive\COMP80004_PhDResearch\RESEARCH\PROJECTS\3_evaluation+validation - ACL 2025\sentiment-breadth-intensity\0.0_corpus_preprocessing\output\natural_lines_targets\anxiety.lines.psych
Saved neutral sentiment sentences for anxiety to c:\Users\naomi\OneDrive\COMP80004_PhDResearch\RESEARCH\PROJECTS\3_evaluation+validation - ACL 2025\sentiment-breadth-intensity\1_sentiment\synthetic\input\baselines\anxiety_neutral

Setup examples to inject in the prompt

In [ ]:
@dataclass
class Example:
    baseline: str
    positive_sentiment: str
    negative_sentiment: str

    def format_for_prompt(self):
        return f"""<baseline>
{self.baseline}
</baseline>
<positive {TARGET_WORD}>
{self.positive_sentiment}
</positive {TARGET_WORD}>
<negative {TARGET_WORD}>
{self.negative_sentiment}
</negative {TARGET_WORD}>
"""

EXAMPLES = [
    Example(
        baseline="In a 24-yr-old female patient with a 12-yr history of kleptomania, it appeared that the behavior was maintained because it reduced anxiety in relevant situations.",
        positive_sentiment="In a 24-yr-old female patient with a 12-yr history of kleptomania, it appeared that the behavior was maintained because it alleviated intense and overwhelming anxiety that pervaded her daily life.",
        negative_sentiment="In a 24-yr-old female patient with a 12-yr history of kleptomania, it appeared that the behavior was maintained because it relieved mild, occasional anxiety in certain situations.",
    ),
    Example(
        baseline="Anxiety, insecurity, and lack of skill in establishing appropriate relationships prevented workers from obtaining adequate bases for evaluation and for effectuation of treatment.",
        positive_sentiment="Intense anxiety, profound insecurity, and a lack of skill in establishing appropriate relationships prevented workers from obtaining adequate bases for evaluation and for effectuation of treatment.",
        negative_sentiment="Mild anxiety, occasional insecurity, and a lack of skill in establishing appropriate relationships prevented workers from obtaining adequate bases for evaluation and for effectuation of treatment.",
    ),
    Example(
        baseline="The relationship between self-reported fear and anxiety was examined in a large sample of normal Australian children and adolescents.",
        positive_sentiment="The relationship between deeply rooted fear and severe anxiety was examined in a large sample of Australian children and adolescents, revealing strong emotional undercurrents.",
        negative_sentiment="The relationship between mild fear and occasional anxiety was examined in a large sample of Australian children and adolescents.",
    )
]

PROMPT_INTRO = f"""In psychology lexicographic research, we define "intensity" as the "extent to which a word refers to more emotionally or referentially intense phenomena". Here we study the intensity of the word "{TARGET_WORD}"
You will be given a sentence with the word "{TARGET_WORD}" in it. You will then be asked to write two new sentences: one where the word "{TARGET_WORD}" is more intense, and one where it is less intense.

"""


In [4]:
@dataclass
class SentenceToModify:
    text: str

    def get_prompt(self):
        prompt = PROMPT_INTRO
        for example in EXAMPLES:
            prompt += example.format_for_prompt()
            prompt += "\n\n"
        
        prompt += f"""<baseline>
{self.text}
</baseline>
"""
        return prompt
    
    def parse_response(self, response: str):
        # get the sentences inside <more {TARGET_WORD}> and <less {TARGET_WORD}>
        positive_variation = response.split(f"<positive {TARGET_WORD}>")[1].split(f"</positive {TARGET_WORD}>")[0].strip()
        negative_variation = response.split(f"<negative {TARGET_WORD}>")[1].split(f"</negative {TARGET_WORD}>")[0].strip()
        return positive_variation, negative_variation

    def get_variations(self) -> list[str]:
        """ Returns a list of two strings: one where the TARGET_WORD is more intense, and one where it is less intense """
        assert TARGET_WORD in self.text, f"TARGET_WORD {TARGET_WORD} not found in text"
        prompt = self.get_prompt()
        res = sm.generate_text(prompt=prompt, llm_provider=PROVIDER, llm_model=MODEL)
        return self.parse_response(res)


In [5]:
sentence = SentenceToModify(text="The anxiety caused by impending surgery left her feeling overwhelmed and vulnerable, affecting her daily life.")
sentence.get_variations()


('The crippling anxiety caused by impending surgery left her feeling utterly overwhelmed and vulnerable, dominating her every waking moment and severely affecting her ability to function.',
 'The mild anxiety caused by impending surgery left her feeling somewhat uneasy and vulnerable, though it only slightly affected her daily life.')